Need to group all songs by genre, sort into three buckets, try to make them even, about 1000 each if possible


In [116]:
import pandas as pd
import numpy as np
from ast import literal_eval
import json

import nltk
from nltk.tokenize import wordpunct_tokenize

from textstat import flesch_reading_ease
from sklearn.feature_extraction.text import TfidfVectorizer

import repetition_matrix

GENRES = ['pop', 'rap', 'rock']

In [45]:
df_chart_genres = pd.read_csv('data/chart_with_genres_14000_full.csv', usecols=['song', 'artist', 'genres'])

/var/folders/j6/6598006d18j528xsd1mzvzgh0000gn/T/ipykernel_59417/2654335718.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chart_genres = pd.read_csv('data/chart_with_genres_14000_full.csv', usecols=['song', 'artist', 'genres'])


In [46]:
df_chart_genres

song                         artist  \
0                                Easy On Me                          Adele   
1                                      Stay  The Kid LAROI & Justin Bieber   
2                             Industry Baby        Lil Nas X & Jack Harlow   
3                                Fancy Like                   Walker Hayes   
4                                Bad Habits                     Ed Sheeran   
...                                     ...                            ...   
330082                        Over And Over                Thurston Harris   
330083                     I Believe In You                Robert & Johnny   
330084                      Little Serenade              The Ames Brothers   
330085  I'll Get By (As Long As I Have You)                 Billy Williams   
330086                                 Judy                Frankie Vaughan   

                                                   genres  
0           ['british soul', 'pop', 'pop soul', 'uk pop']  
1                           ['australian hip hop', 'pop']  
2                               ['lgbtq+ hip hop', 'pop']  
3       ['contemporary country', 'country', 'country r...  
4                                       ['pop', 'uk pop']  
...                                                   ...  
330082                                                NaN  
330083                                                NaN  
330084                                                NaN  
330085                                                NaN  
330086                                                NaN  

[330087 rows x 3 columns]

In [102]:
def decide_genre(genre_list):
    genre_dict = dict(zip(GENRES, [False]*3))
    for genre in genre_list:
        for g in GENRES:
            if g in genre:
                genre_dict[g] = True
    
    present_genres = []
    for genre, present in genre_dict.items():
        if present:
            present_genres.append(genre)
    if len(present_genres) == 1:
        return present_genres[0]
    return None

In [103]:
seen_songs = set()
genre_dict = dict(zip(GENRES, [[],[],[]]))
for song, artist, genre_list in df_chart_genres.to_numpy():
    song = f"{song} by {artist}"
    if song not in seen_songs:
        seen_songs.add(song)
        try:
            genre_list = literal_eval(genre_list)
            decided_genre = decide_genre(genre_list)
            if decided_genre and len(genre_dict[decided_genre]) < 1000:
                genre_dict[decided_genre].append(song)
        except:
            continue

In [104]:
genre_dict

{'pop': ['Easy On Me by Adele',
  'Stay by The Kid LAROI & Justin Bieber',
  'Industry Baby by Lil Nas X & Jack Harlow',
  'Bad Habits by Ed Sheeran',
  'Shivers by Ed Sheeran',
  'Good 4 U by Olivia Rodrigo',
  'Need To Know by Doja Cat',
  'Levitating by Dua Lipa',
  'Essence by Wizkid Featuring Justin Bieber & Tems',
  'Kiss Me More by Doja Cat Featuring SZA',
  'Heat Waves by Glass Animals',
  'You Right by Doja Cat & The Weeknd',
  'Save Your Tears by The Weeknd & Ariana Grande',
  'Traitor by Olivia Rodrigo',
  'My Universe by Coldplay x BTS',
  'Meet Me At Our Spot by THE ANXIETY: WILLOW & Tyler Cole',
  'Montero (Call Me By Your Name) by Lil Nas X',
  'Chasing After You by Ryan Hurd With Maren Morris',
  'Moth To A Flame by Swedish House Mafia & The Weeknd',
  'Thats What I Want by Lil Nas X',
  'Happier Than Ever by Billie Eilish',
  'Better Days by NEIKED X Mae Muller X Polo G',
  'Ghost by Justin Bieber',
  'A-O-K by Tai Verdes',
  'Leave The Door Open by Silk Sonic (Bruno M

In [105]:
with open("data/songs_grouped_by_genre.json", "x") as f:
    json.dump(genre_dict, f)

FileExistsError: [Errno 17] File exists: 'data/songs_grouped_by_genre.json'

repetitiveness, duration, tempo, uniqueness (avg. tf-idf),  readability

In [106]:
df_all_songs = pd.read_json('data/all_songs.json', orient='index')
df_all_songs

song           artist  \
0          Blinding Lights by The Weeknd       The Weeknd   
1         Radioactive by Imagine Dragons  Imagine Dragons   
2                     Sail by AWOLNATION       AWOLNATION   
3           How Do I Live by LeAnn Rimes      LeAnn Rimes   
4          Counting Stars by OneRepublic      OneRepublic   
...                                  ...              ...   
12396    Cold Beer Drinker by Luke Bryan       Luke Bryan   
12397  Who Can I Count On by Patsy Cline      Patsy Cline   
12398        It's All Right by Sam Cooke        Sam Cooke   
12399     Broken Hearted by The Miracles     The Miracles   
12400     Honky-Tonk Girl by Johnny Cash      Johnny Cash   

                                                  chorus  
0      [I said, ooh, I'm blinded by the lights, No, I...  
1      [Whoa-oh, whoa, I'm radioactive, radioactive, ...  
2                         [Sail, Sail, Sail, Sail, Sail]  
3      [How do I live without you? I want to know, Ho...  
4      [Lately, I've been, I've been losin' sleep, Dr...  
...                                                  ...  
12396  [I'm just a cold beer drinker, check out the c...  
12397  [Who can I count on, if I can't count on you?,...  
12398  [Whoa, it's all right (It's all right), It's a...  
12399  [Broken hearted, ooh since we parted, Just say...  
12400  [Breaking hearts to her is just the way of hav...  

[12401 rows x 3 columns]

In [107]:
song2chorus = {song: chorus for song, _, chorus in df_all_songs.to_numpy()}
song2chorus

{'Blinding Lights by The Weeknd': ["I said, ooh, I'm blinded by the lights",
  "No, I can't sleep until I feel your touch",
  "I said, ooh, I'm drowning in the night",
  "Oh, when I'm like this, you're the one I trust",
  'Hey, hey, hey'],
 'Radioactive by Imagine Dragons': ['Whoa-oh, whoa',
  "I'm radioactive, radioactive",
  'Whoa-oh, whoa',
  "I'm radioactive, radioactive"],
 'Sail by AWOLNATION': ['Sail', 'Sail', 'Sail', 'Sail', 'Sail'],
 'How Do I Live by LeAnn Rimes': ['How do I live without you? I want to know',
  'How do I breathe without you if you ever go?',
  'How do I ever, ever survive?',
  'How do I, how do I, oh, how do I live?'],
 'Counting Stars by OneRepublic': ["Lately, I've been, I've been losin' sleep",
  "Dreamin' about the things that we could be",
  "But baby, I've been, I've been prayin' hard",
  "Said no more countin' dollars, we'll be countin' stars",
  "Lately, I've been, I've been losin' sleep",
  "Dreamin' about the things that we could be",
  "But baby, I

In [122]:
def repetitiveness(chorus):
    return repetition_matrix.generate_matrix(chorus)

def readability(chorus):
    text = ' '.join(chorus)
    score = flesch_reading_ease(text)
    if score > 100:
        score = 100
    elif score < 0:
        score = 0
    return score / 100

def flatten(nested_listed):
    return [item for sublist in nested_listed for item in sublist]

# Calculate tf–idf for all words
vectorizer = TfidfVectorizer()
choruses = list(filter(lambda chorus: chorus, [' '.join(song2chorus[song]) if song in song2chorus.keys() else None for song in flatten(genre_dict.values())]))
vectors = vectorizer.fit_transform(choruses)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
dense_list = dense.tolist()
df_tfidf = pd.DataFrame(dense_list, columns=feature_names)

def uniqueness(chorus):
    text = ' '.join(chorus)
    this_chorus_index = choruses.index(text)
    tokens = wordpunct_tokenize(text)
    word_count = len(tokens)
    avg_tfidf = 0
    for token in tokens:
        avg_tfidf += df_tfidf[token][this_chorus_index] / word_count
    return avg_tfidf

def create_dict_list_from_song_list(song_list):
    choruses = [(song, song2chorus[song]) for song in song_list]
    dict_list = [{song: {'repetition': repetitiveness(chorus), 'readability': readability(chorus), 'uniqueness': uniqueness(chorus)}} for song, chorus in choruses]
    return dict_list

/Users/xavierevans/miniforge3/envs/thesis/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [123]:
repetitiveness(['I go to the emerald', 'the emerald city, Esmerelda'])

TypeError: can't multiply sequence by non-int of type 'str'

In [117]:
# genre_dict.values()

In [99]:
# for genre, genre_cluster in genre_dict.items():
#     genre_dict[genre] = [{song: {'repetition': repetitiveness(song2chorus[song]), 'readability': readability(' '.join(song2chorus[song])), 'uniqueness': uniqueness(' '.join(song2chorus[song]))}} if song in song2chorus.keys() else {} for song in genre_cluster]
